# Basic

In [ ]:
import pandas as pd
import os
import numpy as np
import json
from datetime import datetime
from datetime import timedelta
import time

with open("env.json") as f: # input your env file path 
    envs = json.load(f)

# Filter criteria: date, pnum
start_date = datetime.strptime("2023-06-22", "%Y-%m-%d")
end_date = datetime.strptime("2023-07-21", "%Y-%m-%d")
pnum_CSR = [1,2,3,4,
        #5,
        #6
        7,8,9,10,
        #11,
        12,13,14,15,16,17,
        #18,
        19,20,21,22,23] # 5,6,11,18 : manager 


In [3]:
# Function 
def attatch_prefix_condition(df, prefix,exclude_col = ['pnum','start_second','end','date','matching']):
    df.columns = [f"{prefix}_{col}" if col not in exclude_col else col for col in df.columns]
    return df

# Daily survey

In [ ]:
# 1. Daily_survey
DAILY_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","SURVEY_DAILY")
DAILY_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","SURVEY_DAILY")

start = time.time()
print("Start cleaning DAILY data")

# 1.1 Concatenate all DAILY data
start = time.time()
print("Start cleaning DAILY data")

pattern = r'\d{4}-\d{2}-\d{2}' # yyyy-mm-dd
after_daily_df = pd.DataFrame()
before_daily_df = pd.DataFrame()

for folder in os.listdir(DAILY_SOURCE_PATH):
    folder_path = os.path.join(DAILY_SOURCE_PATH, folder)  

    if not os.path.isdir(folder_path):
        continue

    for file in os.listdir(folder_path):        

        # Filter the data within the experiment period
        try:
            file_date_str = re.search(pattern, file).group()
            file_date = datetime.strptime(file_date_str, "%Y-%m-%d")

        except ValueError:  # 날짜 형식이 맞지 않으면 스킵
            continue
        if start_date <= file_date <= end_date:

            file_path = os.path.join(folder_path, file)            
            daily_survey = pd.read_csv(file_path,nrows =1)
            daily_survey['pnum'] = folder
            daily_survey['date'] = file_date_str

            if 'after' in file.lower():  # 파일 이름에 "after"가 포함되어 있는지 확인
                after_daily_df = pd.concat([after_daily_df,daily_survey],axis=0)
            elif 'before' in file.lower():  # 파일 이름에 "before"가 포함되어 있는지 확인
                before_daily_df = pd.concat([before_daily_df,daily_survey],axis=0)
        else : print(file)


# 1.2 Rename column name: 
# before work survey
before_daily_df.rename(columns={
        'timestamp_0' :'before_work_stand_time', ' timestamp_1':'before_work_sleep_time', 
        ' timestamp_2':'before_work_wake_time', ' timestamp_3':'before_work_general_health_time',
        ' timestamp_4':'before_work_stress_time', ' timestamp_5':'before_work_arousal_time', 
        ' timestamp_6':'before_work_valence_time',
        ' answer_3':'before_work_general_health', ' answer_4':'before_work_stress', 
        ' answer_5':'before_work_arousal', ' answer_6': 'before_work_valence',
        },inplace = True) 

# after work survey
after_daily_df.rename(columns={
        'timestamp_0':'after_work_stand_time', 
        ' timestamp_1':'after_work_emotion_exhaution1_time', ' timestamp_2':'after_work_emotion_exhaution2_time',
        ' timestamp_3':'after_work_emotion_exhaution3_time', ' timestamp_4':'after_work_emotion_exhaution4_time',
        ' timestamp_5':'after_work_emotion_exhaution5_time', ' timestamp_6':'after_work_emotional_disharmony1_time',
        ' timestamp_7':'after_work_emotional_disharmony2_time',' timestamp_8':'after_work_emotional_disharmony3_time',
        ' timestamp_9':'after_work_nap_time_time',' timestamp_10':'after_work_general_health_time',
        ' timestamp_11':'after_work_stress_time',' timestamp_12':'after_work_arousal_time',
        ' timestamp_13':'after_work_valence_time',' timestamp_14':'after_work_caffeine',
        ' answer_1': 'after_work_emotional_exhaustion1',' answer_2':'after_work_emotional_exhaustion2',
        ' answer_3':'after_work_emotional_exhaustion3', ' answer_4':'after_work_emotional_exhaustion4', 
        ' answer_5':'after_work_emotional_exhaustion5', ' answer_6': 'after_work_emotional_disharmony1',
        ' answer_7':'after_work_emotional_disharmony2', ' answer_8':'after_work_emotional_disharmony3', 
        ' answer_9':'after_work_nap_time',
        ' answer_10':'after_work_general_health', ' answer_11':'after_work_stress',
        ' answer_12':'after_work_arousal', ' answer_13': 'after_work_valence', 
        ' answer_14' : 'after_work_caffeine'},inplace = True) 
after_daily_df['after_work_emotional_exhaustion'] = after_daily_df[['after_work_emotional_exhaustion1','after_work_emotional_exhaustion2',
                                        'after_work_emotional_exhaustion3','after_work_emotional_exhaustion4',
                                        'after_work_emotional_exhaustion5']].mean(axis=1)
after_daily_df['after_work_emotional_disharmony'] = after_daily_df[['after_work_emotional_disharmony1','after_work_emotional_disharmony2',
                                        'after_work_emotional_disharmony3']].mean(axis=1)

# 1.3 before work feature engineering: sleep time 
# sleep time
before_daily_df['bed_time'] = pd.to_timedelta(before_daily_df[' sleep_hour']*60 + before_daily_df[' sleep_minute'], unit='m')
before_daily_df['wake_time'] = pd.to_timedelta(before_daily_df[' wake_hour']*60 + before_daily_df[' wake_minute'], unit='m')
before_daily_df.loc[before_daily_df[' sleep_hour'] < 8 , 'bed_time'] += pd.Timedelta(days=1) # 이 코드?? 
before_daily_df['wake_time'] += pd.Timedelta(days=1)
before_daily_df['self_reported_sleep_time'] = (before_daily_df['wake_time'] - before_daily_df['bed_time']) / pd.Timedelta(hours=1)

# imputation for abnormal sleep time
over_12 = before_daily_df['self_reported_sleep_time'] > 12
imputation_values = before_daily_df[~over_12].groupby('pnum')['self_reported_sleep_time'].mean()
for p in imputation_values.index:
    before_daily_df.loc[(before_daily_df['pnum'] == p) & over_12, 'self_reported_sleep_time'] = imputation_values[p]

# 1.4. save data
before_daily_df.to_csv(os.path.join(DAILY_DEST_PATH,"daily_before_work.csv"),index=False)
after_daily_df.to_csv(os.path.join(DAILY_DEST_PATH,"daily_after_work.csv"),index=False)

# Call label

## Concatenation

In [ ]:
CALL_LABEL_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","CALL_LABEL")
CALL_LABEL_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","CALL_LABEL")

In [14]:
# Concatenate all call label data
for folder in os.listdir(CALL_LABEL_SOURCE_PATH):
    folder_path = os.path.join(CALL_LABEL_SOURCE_PATH , folder)   
    if not os.path.isdir(folder_path):
        continue
    all_frames = []
    for file in os.listdir(folder_path):      

        # Filter the data within the experiment period
        try:
            file_date_str = file.split('_')[-1].replace('.csv', '')  # 날짜 부분만 추출
            file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
        except ValueError:  # 날짜 형식이 맞지 않으면 스킵
            continue

        # 날짜가 범위 내에 있는지 확인
        if start_date <= file_date <= end_date:
            file_path = os.path.join(folder_path, file)            
            # 파일을 DataFrame으로 읽어서 리스트에 추가
            aftercall = pd.read_csv(file_path)
            all_frames.append(aftercall)
            #print(type(all_frames), type(all_frames[0]), len(all_frames))
        else :
            pass
            # print(file)

    # 모든 파일의 데이터를 하나의 DataFrame으로 합침
    if all_frames: 
        combined_df = pd.concat(all_frames, ignore_index=True)
        # 합친 DataFrame을 변수에 저장
        globals()[f"aftercall_{folder}"] = combined_df
        # print(len(combined_df))
    else : print("Empty: ", folder)

aftercall_all = []  

for i in pnum_CSR:  
    try:
        df = globals()[f'aftercall_{i}'] 
        df = df.rename(columns={'timestamp_0' :'stand_time' , ' timestamp_1':'stress_time', 
                                        ' timestamp_2':'surface_acting_time', ' timestamp_3':'deep_acting_time',
                                        ' timestamp_4':'arousal_time', ' timestamp_5':'valence_time', ' timestamp_6':'eating_time', 
                                        ' timestamp_7': 'drinking_time',
                                        ' answer_1' : 'stress', ' answer_2' : 'surface_acting', ' answer_3': 'deep_acting', ' answer_4':'arousal', ' answer_5':'valence',
                                        ' answer_6':'eating', ' answer_7':'drinking'})
    except KeyError:
        continue 
    df['pnum'] = i  
    # print(df.head(2))
    aftercall_all.append(df)

if aftercall_all:
    aftercall_all = pd.concat(aftercall_all, ignore_index=True)
else:
    print("No data to concatenate.")

Empty:  6
Empty:  18


In [21]:
answer_time = ['stand_time', 'stress_time', 'surface_acting_time', 'deep_acting_time',
       'arousal_time', 'valence_time', 'eating_time', 'drinking_time']
aftercall_all['answer_duration'] = aftercall_all[answer_time].max(axis=1) - aftercall_all[answer_time].min(axis=1)
aftercall_all['answer_start_time'] = aftercall_all[answer_time].min(axis=1)
aftercall_all['answer_start_time'] = pd.to_datetime(aftercall_all['answer_start_time']/1000, unit ='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul').dt.floor('s')
aftercall_all['answer_start_time']= aftercall_all['answer_start_time'].dt.tz_localize(None)

In [22]:
aftercall_all.to_csv(os.path.join(CALL_LABEL_DEST_PATH,"call_label.csv"),index=False)

## Filtering

In [ ]:
aftercall_all = pd.read_csv(os.path.join(CALL_LABEL_DEST_PATH,"call_label.csv"))
callcenter= pd.read_csv(os.path.join(envs['DATA_PATH'],'2_preprocessed','call_log.csv'),parse_dates=['start','start_second','end'])
# callcenter['date'] = callcenter['start_second'].dt.date

In [47]:
# 시간 생성하기
# 다음의 시간중['stress_answer_time', 'SA_answer_time', 'DA_answer_time','arousal_answer_time', 'valence_answer_time', 'eating_answer_time', 'drinking_answer_time']에서 가장 빠른 값을 찾아서,새로운 칼럼, answer_start_time 칼럼에 넣어주기 
answer_time = ['stand_time', 'stress_time', 'surface_acting_time', 'deep_acting_time',
       'arousal_time', 'valence_time', 'eating_time', 'drinking_time']
aftercall_all['answer_duration'] = aftercall_all[answer_time].max(axis=1) - aftercall_all[answer_time].min(axis=1)
aftercall_all['answer_start_time'] = aftercall_all[answer_time].min(axis=1)
aftercall_all['answer_start_time'] = pd.to_datetime(aftercall_all['answer_start_time']/1000, unit ='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul').dt.floor('s')
aftercall_all['answer_start_time']= aftercall_all['answer_start_time'].dt.tz_localize(None)
# aftercall_all['date'] = aftercall_all['answer_start_time'].dt.date

# 중복 데이터 제거
aftercall_all = aftercall_all.drop_duplicates(subset=['pnum','answer_start_time'],keep='first').reset_index(drop=True) # 10개 중복 응답 기록 존재. 따라서 제거

In [8]:
callcenter = callcenter.sort_values(by = ['pnum','start_second']).reset_index(drop=True)
callcenter['after_break'] = callcenter.groupby(['pnum',callcenter['start_second'].dt.date])['break'].shift(-1).fillna(300) # 맨 마지막 call은 5분으로 결정하기
callcenter['afterbreak_end'] = callcenter['end'] + pd.to_timedelta(callcenter['after_break'], unit='s')

In [9]:
# soort
aftercall_all = aftercall_all.sort_values(by = ['pnum','answer_start_time']).reset_index(drop=True) 
callcenter = callcenter.sort_values(by = ['pnum','start_second']).reset_index(drop=True)

# merge
pnum_list = callcenter['pnum'].unique()
abnormal_idx_list = []

aftercall_all_col = ['stand_time', 'stress_time', 'surface_acting_time', 'deep_acting_time',
       'arousal_time', 'valence_time', 'eating_time', 'drinking_time',
       'stress', 'surface_acting', 'deep_acting', 'arousal', 'valence',
       'eating', 'drinking', 'answer_start_time','answer_duration']
callcenter_col = ['start', 'agreement', 'question', 'answer', 'pnum', 'total_duration',
       'complain', 'start_second', 'active_duration', 'in/out', 'matching',
       'end', 'mute', 'date', 'break', 'after_break', 'afterbreak_end']

columns_list =  callcenter_col + aftercall_all_col
merge_final = pd.DataFrame(columns = columns_list)

for pnum in pnum_list:

    # 뽑아낼 pnum의 데이터
    pnum_call_log = callcenter.query('pnum==@pnum')
    pnum_call_label = aftercall_all.query('pnum==@pnum')

    # call log에 맞게 call label 뽑기 
    for idx, row in pnum_call_log.iterrows():
        start = row['end']
        end = row['afterbreak_end']
        
        filtered_call_label = pnum_call_label[(pnum_call_label['answer_start_time']<= end) & (pnum_call_label['answer_start_time']>= start)]
        if len(filtered_call_label)==0:
            add_row = list(row.values)
            add_row.extend([np.nan]*(len(aftercall_all.columns)-1))
            merge_final.loc[len(merge_final)] = add_row
            continue

        if len(filtered_call_label) >= 2:
            abnormal_idx_list.append(list(filtered_call_label.index))    
        add_row = list(row.values)
        add_row.extend(filtered_call_label.iloc[0][aftercall_all_col].values)
        merge_final.loc[len(merge_final)] = add_row

/tmp/ipykernel_43162/3169583281.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merge_final.loc[len(merge_final)] = add_row
/tmp/ipykernel_43162/3169583281.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merge_final.loc[len(merge_final)] = add_row
/tmp/ipykernel_43162/3169583281.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dty

In [13]:
merge_final_cleaned = merge_final[merge_final['answer_start_time'].notnull()]

In [ ]:
# 추출할 column 선정
col = [ 'pnum','start_second','stress', 'surface_acting', 'deep_acting', 'arousal', 'valence']
aftercall_all_matching= merge_final_cleaned[col]

In [ ]:
aftercall_all_matching.to_csv(os.path.join(CALL_LABEL_DEST_PATH ,'call_label.csv'),index=False)

In [48]:
### (폐기) call log 종료 이후 가장 빨리 발생한 call label 매칭 

# # soort
# aftercall_all = aftercall_all.sort_values(by = ['answer_start_time']).reset_index(drop=True) 
# aftercall_all['date'] = aftercall_all['answer_start_time'].dt.date
# callcenter = callcenter.sort_values(by = ['end']).reset_index(drop=True)
# callcenter['date'] = callcenter['start_second'].dt.date

# test = pd.merge_asof(
#                      callcenter, 
#                      aftercall_all,
#                      by=['pnum','date'], 
#                      left_on='end', 
#                      right_on='answer_start_time', 
#                      direction='forward')
# test['time_diff'] = test['answer_start_time'] - test['end']

# one_label_matching  = test.sort_values(by=['pnum','date', 'answer_start_time','time_diff']).drop_duplicates(subset=['pnum','date','answer_start_time'], keep='first')

# display(one_label_matching[one_label_matching.duplicated(subset=['pnum','date', 'end'], keep=False)])
# display(one_label_matching[one_label_matching.duplicated(subset=['pnum','date', 'answer_start_time'], keep=False)])

# one_label_matching = one_label_matching.dropna(subset=['answer_start_time'])

# col = [ 'pnum','start_second','end','stand_time', 'answer_duration', 'answer_start_time' ,
#        'stress_time', 'surface_acting_time', 'deep_acting_time','arousal_time', 'valence_time',
#        'stress', 'surface_acting', 'deep_acting', 'arousal', 'valence']
# aftercall_all_matching_origin = one_label_matching[col]
# aftercall_all_matching_origin.to_csv(os.path.join(CALL_LABEL_DEST_PATH ,'call_label_matching_origin.csv'),index=False)

# Food intake

In [ ]:
FOOD_INTAKE_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","FOOD_INTAKE")

In [ ]:
col = [ 'pnum','start_second','end','answer_start_time','stand_time','eating_time', 'drinking_time','eating', 'drinking']
food_intake= merge_final_cleaned[col]
food_intake = attatch_prefix_condition(food_intake,'food_intake', ['pnum','start_second','end','date','matching',
                                                                   'answer_start_time','stand_time','eating_time',
                                                                   'drinking_time'])

food_intake.to_csv(os.path.join(FOOD_INTAKE_DEST_PATH ,'food_intake.csv'),index=False)

# Call log 

In [ ]:
# matching을 기준으로 할 데이터 선별 
CALL_LOG_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","CALL_LOG")

# call log 불러오기
call_log = pd.read_csv(os.path.join(CALL_LOG_SOURCE_PATH,"call_log.csv"),parse_dates=['start_second','end'],index_col=0)[['pnum','matching','break','date','start_second','end']]

# ms식으로 표현하기 
from datetime import datetime
import pytz
seoul_tz = pytz.timezone('Asia/Seoul')
def find_timestamp(date):
    dt = seoul_tz.localize(datetime.fromtimestamp(date.timestamp()))
    return dt.timestamp()*1000

call_log['timestamp_start'] = call_log['start_second'].apply(lambda x: find_timestamp(x))
call_log['timestamp_end'] = call_log['end'].apply(lambda x: find_timestamp(x))

call_work = call_log.groupby(['pnum','date']).agg({'timestamp_start':['min'], 'timestamp_end':['max']}).reset_index()
call_work.columns = ['pnum','date','work_start','work_end']


# ACC

In [ ]:
ACC_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","ACC")
ACC_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","ACC")

### Concatnation

In [5]:
# data concat

for folder in os.listdir(ACC_SOURCE_PATH):
    num = int(folder)
    if num not in pnum_CSR: # 매니저 제외
        continue
    folder_path = os.path.join(ACC_SOURCE_PATH, folder)  
    print(folder)  

    if not os.path.isdir(folder_path):
        continue
    
    all_frames = []
    for file in os.listdir(folder_path):        
        try:
            file_date_str = file.split('_')[-1].replace('.csv', '')  # 날짜 부분만 추출
            file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
        except ValueError:  
            continue 
        if start_date <= file_date <= end_date:
            file_path = os.path.join(folder_path, file)            
            acc = pd.read_csv(file_path)

            # acc 데이터 work time만 필터링 하기 
            time_info = call_work.query('pnum ==@num and date == @file_date_str')[['work_start','work_end']]
            if time_info.empty: 
                pass
                print(file_date)
            else:
                work_start_num,work_end_num = time_info.iloc[0,:]
                filtered_acc = acc[(acc['Timestamp'] >= work_start_num)&(acc['Timestamp'] <= work_end_num)]
                all_frames.append(filtered_acc)

            #print(type(all_frames), type(all_frames[0]), len(all_frames))
        else : print(file)

    # 모든 파일의 데이터를 하나의 DataFrame으로 합침
    if all_frames: 
        combined_df = pd.concat(all_frames, ignore_index=True)
        # 합친 DataFrame을 변수에 저장
        globals()[f"acc_{folder}"] = combined_df
        # print(len(combined_df))

    else : print("Empty: ", folder)

## 하나로 합치기
pnum_list = []
for i in range(1, 24):
    if i not in pnum_CSR: # 매니저 제외
        continue
    pnum_list.append(i)

acc_all = []  

for i in pnum_list:  
    try:
        df = globals()[f'acc_{i}'] 
    except KeyError:
        continue 
    df['pnum'] = i  
    # print(df.head(2))
    acc_all.append(df)

# 모든 DataFrame을 하나로 합침
if acc_all:
    acc_all = pd.concat(acc_all, ignore_index=True)
else:
    print("No data to concatenate.")

14
2023-06-25 00:00:00
2023-07-16 00:00:00
2023-07-02 00:00:00
Acc_UID_p14_2023-07-22.csv
2023-07-09 00:00:00
Acc_UID_p14_2023-06-21.csv
Acc_UID_p14_2023-07-23.csv
Acc_UID_p14_2023-07-25.csv
Acc_UID_p14_2023-07-24.csv
2023-06-24 00:00:00
21
2023-07-08 00:00:00
2023-07-15 00:00:00
2023-06-23 00:00:00
Acc_UID_p21_2023-07-25.csv
2023-06-24 00:00:00
2023-07-01 00:00:00
Acc_UID_p21_2023-07-22.csv
Acc_UID_p21_2023-06-21.csv
Acc_UID_p21_2023-07-23.csv
2023-07-21 00:00:00
Acc_UID_p21_2023-07-24.csv
2023-07-14 00:00:00
2023-07-07 00:00:00
16
2023-07-09 00:00:00
Acc_UID_p16_2023-07-24.csv
2023-06-22 00:00:00
2023-07-02 00:00:00
Acc_UID_p16_2023-07-22.csv
Acc_UID_p16_2023-07-23.csv
2023-06-25 00:00:00
2023-07-15 00:00:00
2023-07-16 00:00:00
Acc_UID_p16_2023-06-21.csv
Acc_UID_p16_2023-07-25.csv
2
Acc_UID_p02_2023-06-21.csv
2023-07-16 00:00:00
Acc_UID_p02_2023-07-22.csv
2023-06-25 00:00:00
2023-07-15 00:00:00
2023-07-09 00:00:00
2023-07-08 00:00:00
2023-06-24 00:00:00
Acc_UID_p02_2023-07-23.csv
202

### Duplicated data

In [8]:
acc_all['ReadableTimestamp'] = pd.to_datetime(acc_all['Timestamp'], unit ='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
acc_all['date']  = acc_all['ReadableTimestamp'].dt.date # 날짜 정보 만들기
acc_all['date'] = acc_all['date'].astype(str)

In [9]:
duplicated_acc = acc_all[acc_all.duplicated(keep=False)]
# duplicate_group = duplicated_acc.groupby(['pnum'])['date'].unique()

In [10]:
# 중복 기록된 데이터가 어디인지 csv로 저장하기
duplicate_group = duplicated_acc.groupby(['pnum'])['date'].agg(['unique','count'])
duplicate_group.reset_index().to_csv(os.path.join(ACC_DEST_PATH,"acc_oversampling_group.csv"),index=False)

# 중복 데이터가 발생한 날짜에 해당하는 데이터 모두 제거
    # 추가로, 이때 발생한 데이터들은 실제 sampling rate가 200Hz인데 100Hz로 발생함. 
for pnum, dates in duplicated_acc.groupby(['pnum'])['date'].unique().items():
    acc_all = acc_all[~((acc_all['pnum'] == pnum) & acc_all['date'].isin(dates))]

### Abnormal data

In [ ]:
# 모든 value 값이 같은 row 찾기
# acc_all_same = acc_all.loc[(acc_all[' accX'] == acc_all[' accY']) & (acc_all[' accY'] == acc_all[' accZ'])]

In [11]:
# 모든 value가 0인 row 찾기
start_col = acc_all.columns.get_loc(' accX')
end_col = acc_all.columns.get_loc(' accZ')
acc_all_zero = acc_all.loc[(acc_all.iloc[:, start_col:end_col+1]==0).all(axis=1)]

In [13]:
acc_all_zero_df = pd.DataFrame()
for pnum, dates in acc_all_zero.groupby(['pnum'])['date'].unique().items():
    acc_all_zero_df = pd.concat([acc_all_zero_df,acc_all[((acc_all['pnum'] == pnum) & acc_all['date'].isin(dates))]],axis=0)

In [14]:
acc_all_zero.groupby(['pnum'])['date'].count()/acc_all_zero_df.groupby(['pnum'])['date'].count()*100# 수집된 데이터의 90%이상이 0으로 찍힘. 제거 하기. 

pnum
9     100.000000
10     92.940311
12    100.000000
14     92.574751
16    100.000000
17     95.931168
19    100.000000
20    100.000000
21    100.000000
Name: date, dtype: float64

In [15]:
# acc가 모두 0인 값을 갖는 date를 모두 제거하기
acc_all_zero_group = acc_all_zero.groupby(['pnum'])['date'].unique()
for pnum, dates in acc_all_zero_group.items():
    acc_all = acc_all[~((acc_all['pnum'] == pnum) & acc_all['date'].isin(dates))]

In [17]:
# acc_all_zero_group.reset_index().to_csv(os.path.join(ACC_DEST_PATH,"acc_all_zero_group.csv"),index=False)
# acc_all.to_csv(os.path.join(ACC_DEST_PATH,"acc_1.csv"),index=False) # after all_zero_acc_group delete

### Feature engineering

In [6]:
# magnitude
# acc_all['magnitude'] = acc_all.apply(lambda x: np.sqrt(x[' accX']**2 + x[' accY']**2 + x[' accZ']**2), axis=1)
# acc_all.to_csv(os.path.join(ACC_DEST_PATH,"acc.csv"),index=False)

### Winsorization

In [5]:
acc_all= pd.read_csv(os.path.join(ACC_DEST_PATH,"acc.csv"),parse_dates=['ReadableTimestamp','date'])

In [6]:
import pandas as pd
import scipy

def MAD(data, threshold=3.0):
    median = np.median(data)
    mad = np.median(np.abs(data - median))
    outlier_threshold = threshold * mad
    outliers = np.where(np.abs(data - median) > outlier_threshold)[0]
    return outliers

## outlier 처리 
# pnum 별로 acc 이상치 제거하기 
def winsorize(data, lower, upper):
    return data.clip(lower, upper)

In [7]:
pnum_list = acc_all['pnum'].unique()
outlier_acc_all = []

for pnum in pnum_list:
    pnum_acc = acc_all.query('pnum==@pnum')

    for col in [' accX',' accY',' accZ','magnitude']:
        data_col = pnum_acc[col].values

        # 3MAD unpper bound, lower bound 계산하기
        median = np.median(data_col)
        mad = np.median(np.abs(data_col - median))
        lower = median - 3*mad
        upper = median + 3*mad
        
        # lower보다 작은 값은 lower로, upper보다 큰 값은 upper로 대체해주기
        pnum_acc[col] = winsorize(pnum_acc[col],lower,upper)

    outlier_acc_all.append(pnum_acc)

acc_all_outlier = pd.concat(outlier_acc_all, ignore_index=True)

/tmp/ipykernel_2808/3692971024.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnum_acc[col] = winsorize(pnum_acc[col],lower,upper)
/tmp/ipykernel_2808/3692971024.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pnum_acc[col] = winsorize(pnum_acc[col],lower,upper)
/tmp/ipykernel_2808/3692971024.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

### Data Save

In [10]:
acc_all_outlier.to_csv(os.path.join(ACC_DEST_PATH,"acc_win.csv"),index=False)

# ENV

In [ ]:
ENV_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","ENV")
ENV_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","ENV")

### Concatnation

In [5]:
from datetime import datetime

start_date = datetime.strptime("2023-06-22", "%Y-%m-%d")
end_date = datetime.strptime("2023-07-21", "%Y-%m-%d")

for folder in os.listdir(ENV_SOURCE_PATH):
    num = int(folder)
    if num in [5,6,11,18]: # 매니저 제외
        continue
    folder_path = os.path.join(ENV_SOURCE_PATH, folder)  
    print(folder)  
    # 폴더인지 확인
    if not os.path.isdir(folder_path):
        continue
    
    # 각 폴더 내의 모든 파일을 순회
    all_frames = []
    for file in os.listdir(folder_path):        
        # 파일 이름에서 날짜 추출
        #print(file)
        try:
            file_date_str = file.split('_')[-1].replace('.csv', '')  # 날짜 부분만 추출
            file_date = datetime.strptime(file_date_str, "%Y-%m-%d")
            #print(file)
        except ValueError:  # 날짜 형식이 맞지 않으면 스킵
            continue 

        # 날짜가 범위 내에 있는지 확인
        if start_date <= file_date <= end_date:

            file_path = os.path.join(folder_path, file)            
            env = pd.read_csv(file_path)

            # env 데이터 work time만 필터링 하기 
            time_info = call_work.query('pnum ==@num and date == @file_date_str')[['work_start','work_end']]
            if time_info.empty: 
                pass

                print(file_date)
                # # print(work_start_num, work_end_num)
            else:
                work_start_num,work_end_num = time_info.iloc[0,:]
                filtered_env = env[(env['Timestamp'] >= work_start_num)&(env['Timestamp'] <= work_end_num)]
                all_frames.append(filtered_env)

        else : print(file)

    if all_frames: 
        combined_df = pd.concat(all_frames, ignore_index=True)
        globals()[f"env_{folder}"] = combined_df

    else : print("Empty: ", folder)


## 하나로 합치기
pnum_list = []
for i in range(1, 24):
    if i in [5,6,11,18]: # 매니저 제외
        continue
    pnum_list.append(i)

env_all = []  

for i in pnum_list:  
    try:
        df = globals()[f'env_{i}'] 
    except KeyError:
        continue 
    df['pnum'] = i  
    env_all.append(df)

if env_all:
    env_all = pd.concat(env_all, ignore_index=True)
else:
    print("No data to concatenate.")


14
2023-06-24 00:00:00
Env_UID_p14_2023-07-25.csv
Env_UID_p14_2023-06-21.csv
Env_UID_p14_2023-07-22.csv
Env_UID_p14_2023-07-24.csv
2023-07-02 00:00:00
2023-07-09 00:00:00
Env_UID_p14_2023-07-23.csv
2023-07-16 00:00:00
2023-06-25 00:00:00
21
Env_UID_p21_2023-07-25.csv
2023-07-14 00:00:00
Env_UID_p21_2023-07-24.csv
2023-06-24 00:00:00
2023-07-01 00:00:00
Env_UID_p21_2023-07-23.csv
2023-07-08 00:00:00
2023-06-23 00:00:00
2023-07-21 00:00:00
Env_UID_p21_2023-07-22.csv
2023-07-15 00:00:00
Env_UID_p21_2023-06-21.csv
2023-07-07 00:00:00
16
2023-06-25 00:00:00
2023-06-22 00:00:00
Env_UID_p16_2023-07-22.csv
2023-07-09 00:00:00
2023-07-16 00:00:00
Env_UID_p16_2023-06-21.csv
Env_UID_p16_2023-07-23.csv
Env_UID_p16_2023-07-24.csv
Env_UID_p16_2023-07-25.csv
2023-07-02 00:00:00
2023-07-15 00:00:00
2
2023-07-08 00:00:00
2023-06-25 00:00:00
Env_UID_p02_2023-07-22.csv
2023-07-15 00:00:00
2023-07-16 00:00:00
2023-07-01 00:00:00
2023-06-24 00:00:00
Env_UID_p02_2023-07-23.csv
2023-07-09 00:00:00
Env_UID_p0

### Feature engineering

In [6]:
env_all['ReadableTimestamp'] = pd.to_datetime(env_all['Timestamp'], unit ='ms').dt.tz_localize('UTC').dt.tz_convert('Asia/Seoul')
env_all['date']  = env_all['ReadableTimestamp'].dt.date # 날짜 정보 만들기
env_all['date'] = env_all['date'].astype(str)

### Duplicated data

In [15]:
duplicated_env = env_all[env_all.duplicated(keep=False)] # 없음
# duplicate_group = duplicated_env.groupby(['pnum'])['date'].unique()

### Abnormal data

In [7]:
# 모든 value 값이 같은 row 찾기
env_all_same = env_all.loc[(env_all[' Temperature'] == env_all[' Humidity']) & (env_all[' Temperature'] == env_all[' CO2']) & (env_all[' Temperature'] == env_all[' TVOC']) & (env_all[' Humidity'] == env_all[' CO2']) & (env_all[' Humidity'] == env_all[' TVOC']) & (env_all[' TVOC'] == env_all[' CO2'])]

In [8]:
env_all_same.describe()

,Timestamp,Temperature,Humidity,CO2,TVOC,pnum
count,2.230000e+04,22300.0,22300.0,22300.0,22300.0,22300.000000
mean,1.687815e+12,0.0,0.0,0.0,0.0,15.997892
std,2.060058e+08,0.0,0.0,0.0,0.0,0.151661
min,1.687475e+12,0.0,0.0,0.0,0.0,2.000000
25%,1.687589e+12,0.0,0.0,0.0,0.0,16.000000
50%,1.687843e+12,0.0,0.0,0.0,0.0,16.000000
75%,1.688005e+12,0.0,0.0,0.0,0.0,16.000000
max,1.688429e+12,0.0,0.0,0.0,0.0,16.000000


In [9]:
# 모든 value 값이 0인 rwo
start_col = env_all.columns.get_loc(' Temperature')
end_col = env_all.columns.get_loc(' CO2')
env_all_zero = env_all.loc[(env_all.iloc[:, start_col:end_col+1]==0).all(axis=1)]

In [11]:
env_all_zero.head(10)

,Timestamp,Temperature,Humidity,CO2,TVOC,pnum,ReadableTimestamp,date
79123,1688428804549,0.0,0.0,0,0,2,2023-07-04 09:00:04.549000+09:00,2023-07-04
79124,1688428814531,0.0,0.0,0,0,2,2023-07-04 09:00:14.531000+09:00,2023-07-04
401079,1688368338023,0.0,0.0,0,0,9,2023-07-03 16:12:18.023000+09:00,2023-07-03
466427,1687825382436,0.0,0.0,0,0,10,2023-06-27 09:23:02.436000+09:00,2023-06-27
466428,1687825392407,0.0,0.0,0,0,10,2023-06-27 09:23:12.407000+09:00,2023-06-27
815597,1687737904937,0.0,0.0,0,0,16,2023-06-26 09:05:04.937000+09:00,2023-06-26
815598,1687737914938,0.0,0.0,0,0,16,2023-06-26 09:05:14.938000+09:00,2023-06-26
815599,1687737924938,0.0,0.0,0,0,16,2023-06-26 09:05:24.938000+09:00,2023-06-26
815600,1687737934938,0.0,0.0,0,0,16,2023-06-26 09:05:34.938000+09:00,2023-06-26
815601,1687737944939,0.0,0.0,0,0,16,2023-06-26 09:05:44.939000+09:00,2023-06-26


In [23]:
env_all_zero_df = pd.DataFrame()
for pnum, dates in env_all_zero.groupby(['pnum'])['date'].unique().items():
    env_all_zero_df = pd.concat([env_all_zero_df,env_all[((env_all['pnum'] == pnum) & env_all['date'].isin(dates))]],axis=0)

In [24]:
env_all_zero.groupby(['pnum','date'])['Timestamp'].count()/env_all_zero_df.groupby(['pnum','date'])['Timestamp'].count()*100 

pnum  date      
2     2023-07-04      0.062228
9     2023-07-03      0.020109
10    2023-06-27      0.062520
16    2023-06-23    100.000000
      2023-06-24    100.000000
      2023-06-26    100.000000
      2023-06-27    100.000000
      2023-06-28    100.000000
      2023-06-29    100.000000
      2023-06-30    100.000000
Name: Timestamp, dtype: float64

In [28]:
env_all = env_all.loc[~(env_all.iloc[:, start_col:end_col+1]==0).all(axis=1)] # 0으로 기록된 행 모두 제거: 기기 기록 이상으로 판단.

### Data save

In [38]:
env_all.to_csv(os.path.join(ENV_DEST_PATH,"env.csv"),index=False)

# Fitbit

In [ ]:
FITBIT_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","FITBIT")
FITBIT_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","FITBIT")

In [ ]:
# data load
fitbit_hr = pd.read_csv(os.path.join(FITBIT_SOURCE_PATH,"Fitbit_hr.csv"),index_col=0)
fitbit_step = pd.read_csv(os.path.join(FITBIT_SOURCE_PATH,"Fitbit_step.csv"),index_col=0)

# time coloumn 만들기
fitbit_hr.loc[:,'ReadableTimestamp'] = pd.to_datetime(fitbit_hr['date'].astype('str') + ' ' + fitbit_hr['heart-intraday_time'])
fitbit_step.loc[:,'ReadableTimestamp'] = pd.to_datetime(fitbit_step['date'].astype('str')+ ' ' + fitbit_step['steps-intraday_time'])

fitbit_hr['date'] =fitbit_hr['ReadableTimestamp'].dt.date
fitbit_step['date'] = fitbit_step['ReadableTimestamp'].dt.date

###  Filtering

In [7]:
call_work = call_log.groupby(['pnum','date']).agg({'start_second':['min'], 'end':['max']}).reset_index()
call_work.columns = ['pnum','date','work_start','work_end']

In [8]:
# 실험 조건에 해당하는 데이터 선별
fitbit_hr = fitbit_hr.query('pnum in @pnum_CSR')
fitbit_hr  = fitbit_hr[(fitbit_hr['date']>=start_date.date()) & (fitbit_hr['date']<=end_date.date())]

fitbit_step = fitbit_step.query('pnum in @pnum_CSR')
fitbit_step  = fitbit_step[(fitbit_step['date']>=start_date.date()) & (fitbit_step['date']<=end_date.date())]

# Working hour 선별 
result_df_step = pd.DataFrame()
result_df_hr = pd.DataFrame()
# call_log의 각 'pnum'과 'date' 조합에 대해 반복

for idx, row in call_work.iterrows():
 
    pnum = row['pnum']
    first_start_second = row['work_start']
    last_end_second = row['work_end']

    mask = (fitbit_step['pnum'] == pnum) & \
           (fitbit_step['ReadableTimestamp'] >= first_start_second) & \
           (fitbit_step['ReadableTimestamp'] <= last_end_second)
    
    filtered_data = fitbit_step[mask]
    result_df_step = pd.concat([result_df_step, filtered_data], axis=0)

    mask = (fitbit_hr['pnum'] == pnum) & \
           (fitbit_hr['ReadableTimestamp'] >= first_start_second) & \
           (fitbit_hr['ReadableTimestamp'] <= last_end_second)
    filtered_data_hr = fitbit_hr[mask]

    result_df_hr = pd.concat([result_df_hr, filtered_data_hr], axis=0)

### Missing data

#### Step
* step data 사용
* 확인 결과: 2023-07-10에 15번에게 수집이 안된 점 빼고 missing data 없음
* working time안에서는 step은 모두 찍혀있음

In [124]:
check_work = call_work.copy()
check_work['total_minute'] = (pd.to_timedelta(check_work['work_end'] - check_work['work_start']).dt.total_seconds() / 60).astype(int)
check_work['date'] = check_work['date'].astype('str')
check_work['pnum'] = check_work['pnum'].astype('str')

check_step = pd.DataFrame(result_df_step.groupby(['pnum','date'])['ReadableTimestamp'].count().reset_index())
check_step['date'] = check_step['date'].astype('str')
check_step['pnum'] = check_step['pnum'].astype('str')


# 두 데이터프레임을 인덱스로 설정
df1 = check_work.set_index(['pnum', 'date'])[['total_minute']]
df2 = check_step.set_index(['pnum', 'date'])

# 공통된 인덱스만을 기준으로 병합
result = pd.concat([df1, df2], axis=1, join='inner')

# 두 데이터프레임을 인덱스로 설정
df1 = check_work.set_index(['pnum', 'date'])['total_minute']
df2 = check_step.set_index(['pnum', 'date'])

# 공통된 인덱스만을 기준으로 병합
result = pd.concat([df1, df2], axis=1, join='outer')

In [ ]:
result['diff'] = result['ReadableTimestamp'] - result['total_minute']
result['diff'].describe()

count    421.000000
mean       0.508314
std        0.500526
min        0.000000
25%        0.000000
50%        1.000000
75%        1.000000
max        1.000000
Name: diff, dtype: float64

In [131]:
result[result['diff']==1]

total_minute  ReadableTimestamp  diff
pnum date                                             
1    2023-06-22           530              531.0   1.0
     2023-06-26           528              529.0   1.0
     2023-06-27           258              259.0   1.0
     2023-06-29           524              525.0   1.0
     2023-06-30           524              525.0   1.0
...                       ...                ...   ...
23   2023-07-13           519              520.0   1.0
     2023-07-14           264              265.0   1.0
     2023-07-17           534              535.0   1.0
     2023-07-18           534              535.0   1.0
     2023-07-19           530              531.0   1.0

[214 rows x 3 columns]

#### HR
* filtering된 data를 기준으로 3% 정도 1분 이상 기록이 안되는 경우 존재. 

In [ ]:
result_df_hr_sorted = result_df_hr.sort_values(by=['pnum', 'date', 'ReadableTimestamp'])
result_df_hr_sorted['time_diff'] = result_df_hr_sorted.groupby(['pnum', 'date'])['ReadableTimestamp'].diff().dt.total_seconds()

In [11]:
result_df_hr_sorted[result_df_hr_sorted['time_diff']>60]['time_diff'].describe()

count      464.000000
mean       494.034483
std       1444.203323
min         61.000000
25%         80.000000
50%        120.500000
75%        294.250000
max      22650.000000
Name: time_diff, dtype: float64

In [167]:
# HR 값 확인 # 이상치 없는 듯 보임
result_df_hr.groupby(['pnum'])['heart-intraday_value'].describe()

,count,mean,std,min,25%,50%,75%,max
pnum,,,,,,,,
1,63641.0,80.033343,10.468105,53.0,73.0,78.0,85.0,164.0
2,77822.0,79.673447,8.631932,57.0,74.0,78.0,84.0,144.0
3,78946.0,96.693309,9.750883,67.0,90.0,96.0,103.0,140.0
4,76372.0,76.460954,9.250401,51.0,70.0,75.0,81.0,135.0
7,77496.0,79.579656,7.975906,52.0,75.0,79.0,83.0,139.0
8,50791.0,72.247583,7.502819,52.0,67.0,72.0,76.0,126.0
9,90298.0,89.065727,9.561570,63.0,83.0,88.0,94.0,147.0
10,71713.0,89.031473,7.593662,66.0,84.0,88.0,93.0,137.0
12,84293.0,86.593406,9.417198,60.0,80.0,86.0,92.0,148.0


#### HR and Step
* (논문내용) Step data are always collected by the inertial measurement unit (IMU) sensor, and Fitbit
records steps as long as it is turned on. The Fitbit device does not distinguish whether the patient is stationary
or not wearing the device. In both cases, Fitbit reports the step count as "0". The heart rate data can suffer a
significant missing rate, because they are measured by the PPG sensor which requires proper contact with the
wrist.[https://dl.acm.org/doi/pdf/10.1145/3534578]

In [255]:
result_df_hr['ReadableTimestamp_rounded'] = result_df_hr['ReadableTimestamp'].dt.floor('T')
result_df_step['ReadableTimestamp_rounded'] = result_df_step['ReadableTimestamp'].dt.floor('T')


/tmp/ipykernel_30336/461230773.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  result_df_hr['ReadableTimestamp_rounded'] = result_df_hr['ReadableTimestamp'].dt.floor('T')
/tmp/ipykernel_30336/461230773.py:2: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  result_df_step['ReadableTimestamp_rounded'] = result_df_step['ReadableTimestamp'].dt.floor('T')


In [ ]:
result_df_hr['data_type'] =1
result_df_step['data_type'] =1

In [ ]:
filtered_rows = []

# fitbit_step의 각 행에 대해 확인
for index, step_row in result_df_step.iterrows():
    pnum = step_row['pnum']
    step_time = step_row['ReadableTimestamp_rounded']

    # 동일한 pnum과 시간을 가진 result_df_hr의 행 찾기
    hr_rows = result_df_hr[(result_df_hr['pnum'] == pnum) & (result_df_hr['ReadableTimestamp_rounded'] == step_time)]

    # 걸음 수가 0이 아니고, 심박수 기록이 없는 경우 1) 착용 불량, 2) not wearing, 3) 다른 곳에 두었을 경우 
    if step_row['steps-intraday_value'] == 0 and (hr_rows.empty):
        # print(step_row)/
        filtered_rows.append(step_row)

# 필터링된 결과를 DataFrame으로 변환
filtered_data = pd.DataFrame(filtered_rows) # 30개 

In [ ]:
import numpy as np

# pnum과 ReadableTimestamp를 반복하면서 조건에 맞는 'steps-intraday_value' 값을 np.nan으로 변경
for pnum, ReadableTimestamp in filtered_data[['pnum', 'ReadableTimestamp']].values:
    # result_df_step.loc[
    #     (result_df_step['ReadableTimestamp'] == ReadableTimestamp) & 
    #     (result_df_step['pnum'] == pnum), 
    #     'steps-intraday_value'
    # ] = np.nan 
    result_df_step.loc[
        (result_df_step['ReadableTimestamp'] == ReadableTimestamp) & 
        (result_df_step['pnum'] == pnum), 
        'data_type'
    ] = 2 # HR 데이터를 기반으로 nan 값 처리한 것

result_df_step_filter = result_df_step.copy()
print(result_df_step_filter.shape)

(214273, 7)


### Outlier Data
We created an Isolation Forest using the scikit-learn
library [70], with 250 trees, and randomly partitioned samples into each tree. The maximum number of features
per tree was set to the length of the feature space [https://dl.acm.org/doi/pdf/10.1145/3463528]

#### Step

In [ ]:
from sklearn.ensemble import IsolationForest
# Isolation Forest 모델 정의
iso = IsolationForest(n_estimators=250, contamination=0.002)  # 하이퍼 파라미터 정해야 함 

# 모델 학습
result_df_step_filter_anomaly = result_df_step_filter[result_df_step_filter['steps-intraday_value'].notnull()]
result_df_step_filter_anomaly['anomaly'] = iso.fit_predict(result_df_step_filter_anomaly[['steps-intraday_value']]) 

# -1은 이상치로 판별된 행
anomalies = result_df_step_filter_anomaly[result_df_step_filter_anomaly['anomaly'] == -1]

# data concat
result_df_step_filter = result_df_step_filter[['ReadableTimestamp','steps-intraday_value', 'date', 'pnum','data_type']]
result_df_step_filter_anomaly = result_df_step_filter_anomaly[['ReadableTimestamp','steps-intraday_value', 'date', 'pnum','anomaly']]

result_df_step_final = pd.concat([result_df_step_filter.set_index(['ReadableTimestamp','steps-intraday_value', 'date', 'pnum']),result_df_step_filter_anomaly.set_index(['ReadableTimestamp','steps-intraday_value', 'date', 'pnum'])],axis=1).reset_index()

# anomaly 값이 -1인 조건을 만족하는 행들에 대해 처리
result_df_step_final.loc[result_df_step_final['anomaly'] == -1, 'data_type'] = 3 # isolation forest 결과
# result_df_step_final.loc[result_df_step_final['anomaly'] == -1, 'steps-intraday_value'] = np.nan

In [ ]:
# 정상 카운트를 벗어나는 경우를 측정하기 위하여 break 데이터만 추출 
# call_log의 각 행에 대해 필터링 수행 <- 
for _, row in call_log.iterrows():
    mask = (result_df_step_final['pnum'] == row['pnum']) & \
           (result_df_step_final['ReadableTimestamp'] >= row['start_second']) & \
           (result_df_step_final['ReadableTimestamp'] <= row['end'])& \
           (result_df_step_final['steps-intraday_value'] != 0)
    result_df_step_final.loc[mask,'data_type'] = 4 # 1,2,3과 겹치는 경우 없어서 그냥 이렇게 함
    

### Data save

In [244]:
# 데이터 저장
result_df_step_final.to_csv(os.path.join(FITBIT_DEST_PATH ,"fitbit_step.csv"),index=False)
result_df_hr[['heart-intraday_value', 'date', 'pnum',
       'ReadableTimestamp','data_type']].to_csv(os.path.join(FITBIT_DEST_PATH ,"fitbit_hr.csv"),index=False)

# Individual factor

In [ ]:
INDI_SOURCE_PATH = os.path.join(envs['DATA_PATH'],"1_raw","SURVEY_PRE")
INDI_DEST_PATH = os.path.join(envs['DATA_PATH'],"2_preprocessed","INDIVID_FACTOR")

In [5]:
individual_factor_df = pd.read_csv(os.path.join(INDI_SOURCE_PATH,"preservey(0620)_revised.csv"))
individual_factor_df= individual_factor_df[['피험자 번호','성별','나이(만 나이)','최종학력 ','주로 사용하는 손은 어느쪽인가요?',"본 실험에 참여하게 된 계기는 무엇인가요? ","(콜센터 근무 이력이 있으신 경우) 콜센터에 얼마나 근무하셨는지, 회사의 종류에 상관없이 총 근무년수를 적어주세요. (2020년 만 3년 근무 후 휴직하고 2022년에 복직해서 만 1년 근무한 경우  --> '4'로 기입) ","(콜센터 근무 이력이 있으신 경우) 콜센터 회사를 이직한 횟수를 적어주세요. (콜센터와 무관한 회사로의 이직 및 경력 제외) ","(콜센터 근무 이력이 있으신 경우) 회사를 이직한 횟수를 적어주세요. (콜센터와 무관한 회사로의 이직 및 경력 포함) "]]

# rename
individual_factor_df.columns = ['pnum', 'gender', 'age', 'education',"major_hand","engage_motivation","career","turnover_call_center","turnover_all"]

# filter
individual_factor_df = individual_factor_df.query('pnum in @pnum_CSR')


# missing data preprocessing
individual_factor_df['gender'] = individual_factor_df['gender'].apply(lambda x: 1 if x=="여성" else 0)
individual_factor_df['career'] = individual_factor_df['career'].fillna(0)
individual_factor_df['turnover_call_center'] = individual_factor_df['turnover_call_center'].apply(
    lambda x: 1 if x == "1번" else 3 if x == "3회" else 0 if x == "없음" else 0 if x=="무" else x
)

individual_factor_df['turnover_all'] = individual_factor_df['turnover_all'].apply(
    lambda x: 1 if x == "1번" else 3 if x == "없음(콜센터 외의 회사이직 횟수 3회 이상)" else 0 if x == "없음" else 3 if x=="3회" else x
)

individual_factor_df['turnover_call_center'] = individual_factor_df['turnover_call_center'].astype('int')
individual_factor_df['turnover_all'] = individual_factor_df['turnover_all'].astype('int')


In [6]:
individual_factor_df.to_csv(os.path.join(INDI_DEST_PATH,"individual_factor.csv"),index=False)